# 使用 peft 微调大模型

环境
- Apple M4 Mac mini
- Python 3.11.9
- torch 2.9.0
- transformers 4.57.1

参考链接：
- https://huggingface.co/docs/peft/quicktour

## 一、微调技术介绍

**PEFT 是什么**

PEFT = Parameter-Efficient Fine-Tuning，中文叫 **参数高效微调**。

它是由 **Hugging Face** 推出的一个 Python 库，用来在不修改大模型全部参数的前提下，对模型进行轻量级微调。


---

**为什么要用 PEFT？**

以一个 70 亿参数的模型（7B）为例：

* 全参数微调需要几十 GB 显存；
* 训练时间长；
* 模型保存也占很多空间。

而 PEFT 通过“只训练部分参数”的方式（比如 LoRA 层），可以：

* 让微调在 **消费级 GPU（如 RTX 4090 或 A100 40G）** 上完成；
* 大幅节省内存和显存；
* 保留模型的原始能力；
* 输出一个非常小的“适配器权重”（几百 MB 甚至更少）。

---

**PEFT 的核心思想**

PEFT 提供了几种 **参数高效微调方法** 的统一接口：

| 方法                             | 原理                    | 适用场景         |
| ------------------------------ | --------------------- | ------------ |
| **LoRA** (Low-Rank Adaptation) | 在模型的线性层插入可训练的低秩矩阵     | 最常用；大模型任务通用  |
| **Prefix Tuning**              | 给模型输入加上可训练的“提示向量”     | 少样本学习、NLP    |
| **Prompt Tuning**              | 学习一小段 token embedding | Prompt 工程自动化 |
| **P-Tuning v2**                | 比 Prompt Tuning 更强的版本 | 复杂任务         |
| **AdaLoRA**                    | 动态调整 LoRA rank 大小     | 提高效率与性能平衡    |

## 二、准备工作

**确认依赖库版本**

In [3]:
import numpy as np, torch, transformers
np.__version__, torch.__version__, transformers.__version__

('2.3.2', '2.9.0', '4.57.1')

**关闭多线程分词，避免多进程冲突警告**。

Hugging Face 的 tokenizers 库在后台使用 Rust 多线程加速（并行分词）。
但是，当 Python 的 multiprocessing 在已经启用了并行的情况下调用 fork() 时，会触发这个警告，以避免潜在的死锁。

In [5]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

**允许 MPS 不支持的算子自动 fallback 到 CPU**

In [6]:
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1" 

**定义 device，在 MAC 上使用 mps**

In [7]:
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

**开启训练详细日志，以便查看训练细节**

In [8]:
import logging
logging.getLogger("transformers.trainer").setLevel(logging.INFO)

## 三、基础模型测试

用默认的 bigscience/mt0 模型测试一下英语翻译成中文的效果

`bigscience/mt0-large` 介绍

| 项目       | 内容                                                                               |
| -------- | -------------------------------------------------------------------------------- |
| **模型名称** | `bigscience/mt0-large`                                                           |
| **来源**   | [BigScience Project (Hugging Face)](https://huggingface.co/bigscience/mt0-large) |
| **模型类型** | **T5 结构（Encoder-Decoder）多语言序列到序列模型**                                             |
| **参数量**  | 1.2B 参数（中型）                                                                      |
| **基础模型** | MT5（Multilingual T5）                                                             |
| **支持语言** | 100 多种语言，包括英语、中文、法语、西班牙语、阿拉伯语、俄语等                                                |
| **任务类型** | Text-to-text（可用于翻译、分类、摘要、问答、指令生成等）                                               |
| **训练方式** | 指令微调（Instruction-tuned），在多语言指令数据上训练                                              |
| **特点**   | - 强多语言理解能力                                                                       |
|          | - 已经进行过指令调优（比原始 mT5 更易用）                                                         |
|          | - 支持通用文本生成任务                                                                     |




`mt0-large` 太大，建议在 Mac 上从小版本开始微调：

| 模型名称                   | 参数量   | 是否多语言 | 适合 Mac?  |
| ---------------------- | ----- | ----- | -------- |
| `bigscience/mt0-small` | ~300M | ✅     | ✅ 强烈推荐   |
| `bigscience/mt0-base`  | ~580M | ✅     | ✅ 可行     |
| `bigscience/mt0-large` | 1.2B  | ✅     | ⚠️ 可能超内存 |
| `bigscience/mt0-xl`    | 3.7B  | ✅     | ❌ 不建议    |

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "bigscience/mt0-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)

text = '''Translate English to Chinese: 
The difficult access to these older sites for displaced persons gives rise to 
concerns about the living conditions of the people who are residents there, 
in particular widows, the elderly and children, 
who often live in intolerable hardship.'''
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=1024)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

对于那些被逐出的老人来说,这些老城的人口稀少,尤其是老人、老人和孩子,通常会面临严重的社会问题。


## 四、微调训练

### 1、创建 LoRa 配置

每个 PEFT 方法都由一个 PeftConfig 类定义，该类存储构建 PeftModel 所需的所有重要参数。

例如，要使用 LoRa 进行训练，请加载并创建一个 LoraConfig 类。
- task_type ：要训练的任务，要与模型类型匹配（本例中为序列到序列语言建模）
  - `SEQ_CLS` → 序列分类（BERT 类）
  - `CAUSAL_LM` → 自回归语言模型（GPT 类）
  - `SEQ_2_SEQ_LM` → 序列到序列（T5、BART 类）
- inference_mode ：是否使用该模型进行推理
- r ：低秩矩阵的维数，正常值 8， 小内存设置成 4
- lora_alpha ：低秩矩阵的缩放因子，正常值 32， 小内存设置成 8
- lora_dropout ：LoRa 层的 dropout 概率

In [10]:
from peft import LoraConfig, TaskType
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM,  inference_mode=False, r=4, lora_alpha=8, lora_dropout=0.05)

### 2、创建 PEFT 模型
LoraConfig 设置完成后，使用 get_peft_model() 函数创建一个 PeftModel 。

该函数接受一个基础模型（可以从 Transformers 库加载）和一个 LoraConfig， 其中包含用于配置模型以使用 LoRA 进行训练的参数。

In [11]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 442,368 || all params: 582,843,648 || trainable%: 0.0759


可以看到总共 582M 参数，可训练的参数有 7%。

现在你可以使用 Transformers Trainer 、Accelerate 或任何自定义的 PyTorch 训练循环来训练模型了。

### 3、准备训练数据集

opus100 是一个支持 100 种语言对翻译的数据集，我们选择英文翻译成中文的子集，并且为了降低内存使用量，我们只使用 1% 的数据。

In [12]:
from datasets import load_dataset
dataset = load_dataset("opus100", "en-zh", split="train[:1%]").train_test_split(test_size=0.1)

查看数据集

In [13]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 1000
    })
})

In [14]:
dataset["train"][:3]

{'translation': [{'en': 'I order you!', 'zh': '我命令你回来！'},
  {'en': 'Look, if she comes here just let me know or, or get her to ring me or, or something',
   'zh': '就通知我,或請她打給我'},
  {'en': 'Consequently, it occurs to the Nigerian delegation that it might be wise for us not to proceed to take action on the present draft amendment, and I would therefore urge my Canadian friends and the other sponsors of that draft to tarry a little and let us, outside the premises of this grand Hall — perhaps in one of our other smaller rooms, under one of the Vice-Presidents of the General Assembly — meet and further reflect on this draft instead of forcing it to a vote.',
   'zh': '当然，在尼日利亚代表团看来，我们或许以不就当前的修订草案开始采取行动为宜，为此，我敦促加拿大的朋友和该草案其他提案国再等一等，让我们在此大会议厅外、或许在我们其他较小的会议室里，在大会副主席的主持下聚在一起，对草案再作考量，而不是硬着头皮进行表决。'}]}

**其它数据集介绍**

| 任务类型 | 推荐数据集                                        | 语言支持 |
| ---- | -------------------------------------------- | ---- |
| 分类   | XNLI, PAWS-X, Amazon Reviews                 | 多语言  |
| 翻译   | OPUS100, Flores200, WMT                      | 多语言  |
| 摘要   | MLSUM, WikiLingua                            | 多语言  |
| 问答   | TyDi QA, XQuAD, SQuAD                        | 多语言  |
| 指令   | FLAN, SuperNI, OpenOrca, Alpaca-Multilingual | 多语言  |


### 4、对数据进行预处理

定义预处理函数：把原始文本构造成给模型的 prompt，并进行分词

In [15]:
def preprocess(examples):
    # 构造输入 prompt（mt0 是 instruction-style，因此加上任务说明）   
    inputs = [f"translate English to Chinese: {ex['en']}" for ex in examples["translation"]] # 把每个英文句子包装成翻译指令
    targets = [ex["zh"] for ex in examples["translation"]]

    # 使用 tokenizer 对 inputs 做编码（截断 + padding 到 max_length）
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=128)  # 编码输入，统一长度

    # 对目标文本进行编码，得到 labels
    labels = tokenizer(targets, truncation=True, padding="max_length", max_length=128)  # 编码目标文本

    # 将 labels 的 pad token 替换为 -100，这是 transformers 损失函数忽略的标记
    # 这样在计算交叉熵时，对 pad 部分不会贡献损失
    labels_input_ids = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label] for label in labels["input_ids"]
    ]  # 将 labels 中的 pad id 转为 -100

    # 把处理好的 labels 放回 model_inputs 字典，Trainer 会读取 "labels" 字段用于计算 loss
    model_inputs["labels"] = labels_input_ids  # 为返回的数据添加 labels

    # 返回最终的编码后的输入字典（包含 input_ids, attention_mask, labels）
    return model_inputs  # 返回每个样本的字典

把预处理函数映射到整个数据集（batched=True 表示按批次处理以加速）

In [16]:
tokenized_datasets = dataset.map(preprocess, batched=True)  # 对 dataset 的每个 split 应用 preprocess 函数

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

查看预处理后的数据集

In [17]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [18]:
example = tokenized_datasets["train"][0]
print("Input:", tokenizer.decode(example["input_ids"], skip_special_tokens=True))
print("Label:", tokenizer.decode([id for id in example["labels"] if id != -100], skip_special_tokens=True))

Input: translate English to Chinese: I order you!
Label: 我命令你回来!


### 5、定义评测指标

`SacreBLEU` 是一个标准化的 **BLEU（Bilingual Evaluation Understudy）** 实现，由 **Matt Post (2018)** 提出。
它的目标是让 BLEU 评测更 **可复现（reproducible）**、更 **一致（consistent）**。

传统 BLEU 实现存在很多不一致的问题（例如 tokenization、大小写处理等），导致论文之间的 BLEU 分数无法直接比较。
`sacrebleu` 通过**内置标准化的分词器、预处理规则、版本控制**等手段解决了这一痛点。

`evaluate` 是 Hugging Face evaluate 库，用于加载评测指标

In [21]:
import evaluate
metric = evaluate.load("sacrebleu") 

定义 compute_metrics，用于 Trainer 在 eval 时计算指标

In [20]:
def compute_metrics(eval_pred):
    # eval_pred 的典型形式是 (predictions, labels)；这里 predictions 是生成器输出的 token id 矩阵
    preds, labels = eval_pred  # 解包预测结果和标签

    # 如果 predictions 是 logits（非生成模式），则需要取 argmax；但 predict_with_generate=True 时 preds 已是 token id
    if isinstance(preds, tuple):  # 万一 preds 是 tuple（某些版本返回 logits 等），我们取第一个
        preds = preds[0]

    # 将预测中的 -100（如果有的话）替换为 tokenizer.pad_token_id，便于 decode
    # 但正常生成结果里不应该有 -100，这里是稳健处理
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)  # 将 -100 换为 pad id（以便 decode）

    # 把预测和真实标签解码成字符串（跳过 special tokens）
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)  # 解码预测文本
    # labels 里仍可能有 -100，需要把它们换成 pad_token_id 再解码
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)  # 将 labels 中的 -100 替回 pad id
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)  # 解码真实文本

    # sacrebleu 的 compute 需要 references 是 list of list 的形式：[[ref1], [ref2], ...]
    references = [[l] for l in decoded_labels]  # 为每个样本创建单一引用列表

    # 调用 sacrebleu 指标进行计算并返回结果字典（sacrebleu 返回 'score' 键表示 BLEU）
    result = metric.compute(predictions=decoded_preds, references=references)  # 计算 sacrebleu 分数
    # 将结果包装成 dict 返回，Trainer 会把它显示在 eval 输出里
    return {"bleu": result["score"]}  # 返回一个包含 BLEU 分数的字典

**其他常见机器翻译 / 文本生成评测指标**

| 指标                              | 类型               | 核心思想                | 适用场景         | 优点         | 缺点           |
| ------------------------------- | ---------------- | ------------------- | ------------ | ---------- | ------------ |
| **BLEU / SacreBLEU**            | 基于词汇匹配           | n-gram 精度 + 长度惩罚    | 机器翻译、文本生成    | 标准、稳定、可复现  | 忽略语义，不敏感于同义词 |
| **CHRF / CHRF++**               | 基于字符匹配           | 计算字符 n-gram F-score | 低资源语言、形态丰富语言 | 对拼写变化更鲁棒   | 难以解释、不直观     |
| **METEOR**                      | 基于词匹配 + 词形 + 同义词 | 考虑词形还原与同义词匹配        | 翻译评估         | 对语义变化更敏感   | 实现复杂、较慢      |
| **TER (Translation Edit Rate)** | 基于编辑距离           | 最少编辑操作数 / 参考长度      | 机器翻译         | 易解释（与人工一致） | 不考虑意义，只看编辑操作 |
| **ROUGE**                       | 基于召回率的 n-gram 匹配 | 比较生成文本与参考摘要的重叠      | 摘要生成         | 标准指标，通用性高  | 对词序、语义不敏感    |
| **BERTScore**                   | 语义级指标            | 基于 BERT 词向量相似度      | 文本生成、翻译、对话   | 考虑语义相似     | 计算成本高，需要模型   |
| **COMET / BLEURT**              | 学习型指标            | 使用神经网络学习人工评估分数      | 高级 MT 评测     | 与人类一致性高    | 需 GPU，训练依赖大  |

---

**选型建议（适用场景）**

| 场景               | 推荐指标                             | 理由           |
| ---------------- | -------------------------------- | ------------ |
| **机器翻译标准基线**     | `sacrebleu`                      | 标准、易对比、被广泛使用 |
| **形态复杂或低资源语言**   | `chrf` / `chrf++`                | 字符级鲁棒        |
| **生成摘要/对话**      | `rouge` + `bertscore`            | 兼顾召回与语义      |
| **高质量语义评测**      | `bertscore` / `comet` / `bleurt` | 与人类评判最一致     |
| **人工评测替代（神经模型）** | `comet`                          | 最新一代、语义敏感    |

---

**总结**

| 类别         | 示例指标                            | 关键特点            |
| ---------- | ------------------------------- | --------------- |
| **基于表面匹配** | BLEU / SacreBLEU / ROUGE / CHRF | 快速、解释性强，但不懂语义   |
| **基于语义嵌入** | BERTScore / COMET / BLEURT      | 能理解同义、语境，接近人工判断 |
| **基于编辑距离** | TER                             | 可解释性强，但不智能      |
| **综合指标**   | COMETKiwi / GEMBA               | 结合语言模型和上下文语义    |


### 6、定义训练参数

为了能在单机上进行微调，有一些参数要特殊设置

| 优化项        | 原因                         | 建议值          |
| ---------- | -------------------------- | ------------ |
| 模型大小       | 减少显存占用                     | `mt0-base`   |
| Batch size | 避免内存溢出                     | 1            |
| LoRA 参数    | 进一步轻量化                     | r=4, alpha=8 |
| 精度         | MPS 不支持 fp16               | 使用默认 fp32    |
| 数据量        | 控制训练规模                     | 1~5% 数据即可测试  |

In [23]:
from transformers import Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./mt0-lora",                  # 模型输出和检查点保存目录
    per_device_train_batch_size=1,            # 每个 GPU/设备上的训练 batch size（根据显存调整）, Mac上必须非常小，否则内存爆
    per_device_eval_batch_size=1,             # 验证时的 batch size
    gradient_accumulation_steps=8,            # 把 batch size 设为 1，用梯度累积模拟更大 batch 模拟 batch size=8
    learning_rate=5e-5,                       # 学习率，默认5e-5，内存小调低学习率
    num_train_epochs=1,                       # 训练轮次，#先跑1个epoch测试稳定性
    eval_strategy="epoch",                    # 评估频率（这里设置为每个 epoch 评估一次）
    predict_with_generate=True,               # 在评估时使用 generate() 来产生文本（必需用于语言生成任务）
    save_strategy="epoch",                    # epoch 表示每个 epoch 保存一次checkpoint），不能设置为 no 来降低内存，必须和 eval_strategy 一致
    logging_strategy="steps",                 # 日志策略（按 steps）
    logging_steps=10,                         # 每多少 step 打印一次日志
    fp16=False,                               # 如果支持则开启半精度训练以节省显存（需要 CUDA + 支持）#  MPS 不支持 fp16
    load_best_model_at_end=True,              # 在训练结束加载最佳验证结果对应的模型
    dataloader_pin_memory=False,              # 在 CUDA 上能让 DataLoader 更快地把数据拷贝到 GPU。MPS 不支持
    metric_for_best_model="bleu",             # 根据哪个指标判断最佳模型
    report_to="none",                         # 不向wandb等发送日志
    log_level="info",                         # 输出日志到控制台
    log_level_replica="info",                 # 对多进程安全
    disable_tqdm=False,                       # 禁用进度条，防止日志被覆盖
)

### 7、创建训练器

这里 eval_dataset 尝试使用 validation，若数据集没有 validation split，会使用 test。在真实训练里建议准备独立的 validation。

In [24]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,                              # 注入（已包装 LoRA 的）模型
    args=training_args,                       # 训练参数
    train_dataset=tokenized_datasets["train"],# 训练集
    eval_dataset=tokenized_datasets["validation"] if "validation" in tokenized_datasets else tokenized_datasets["test"], 
    processing_class=tokenizer,               # tokenizer（用于 decode/generation 时）
    compute_metrics=compute_metrics,          # 评估时要计算的指标函数
)

添加打印日志的回调，一般来说已经添加了，这里为了显示训练过程中的细节，显式的添加一下

In [25]:
from transformers.trainer_callback import PrinterCallback
trainer.add_callback(PrinterCallback)

### 8、启动训练 

运行训练过程；训练期间会按 training_args 指定的策略保存 checkpoint 并在 eval 时计算指标

In [26]:
trainer.train()

The following columns in the Training set don't have a corresponding argument in `PeftModelForSeq2SeqLM.forward` and have been ignored: translation. If translation are not expected by `PeftModelForSeq2SeqLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 9,000
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 8
  Total optimization steps = 1,125
  Number of trainable parameters = 442,368


Epoch,Training Loss,Validation Loss,Bleu
1,3.537800,3.218850,10.728505


{'loss': 4.0011, 'grad_norm': 0.2660370469093323, 'learning_rate': 4.96e-05, 'epoch': 0.008888888888888889}
{'loss': 4.116, 'grad_norm': 0.34418511390686035, 'learning_rate': 4.915555555555556e-05, 'epoch': 0.017777777777777778}
{'loss': 4.2667, 'grad_norm': 0.35320037603378296, 'learning_rate': 4.871111111111111e-05, 'epoch': 0.02666666666666667}
{'loss': 3.8212, 'grad_norm': 0.5087679028511047, 'learning_rate': 4.826666666666667e-05, 'epoch': 0.035555555555555556}
{'loss': 4.1358, 'grad_norm': 1.3656669855117798, 'learning_rate': 4.782222222222222e-05, 'epoch': 0.044444444444444446}
{'loss': 4.0941, 'grad_norm': 0.6689131855964661, 'learning_rate': 4.737777777777778e-05, 'epoch': 0.05333333333333334}
{'loss': 4.158, 'grad_norm': 0.5645339488983154, 'learning_rate': 4.6933333333333333e-05, 'epoch': 0.06222222222222222}
{'loss': 3.681, 'grad_norm': 0.4290207326412201, 'learning_rate': 4.648888888888889e-05, 'epoch': 0.07111111111111111}
{'loss': 3.9938, 'grad_norm': 0.39746496081352234

The following columns in the Evaluation set don't have a corresponding argument in `PeftModelForSeq2SeqLM.forward` and have been ignored: translation. If translation are not expected by `PeftModelForSeq2SeqLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1000
  Batch size = 1


{'eval_loss': 3.2188501358032227, 'eval_bleu': 10.728504525135813, 'eval_runtime': 370.3786, 'eval_samples_per_second': 2.7, 'eval_steps_per_second': 2.7, 'epoch': 1.0}


Saving model checkpoint to ./mt0-lora/checkpoint-1125
loading configuration file config.json from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/config.json
Model config MT5Config {
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "dtype": "float32",
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.57.1",
  "us

{'train_runtime': 1865.4904, 'train_samples_per_second': 4.824, 'train_steps_per_second': 0.603, 'train_loss': 3.748817679511176, 'epoch': 1.0}


TrainOutput(global_step=1125, training_loss=3.748817679511176, metrics={'train_runtime': 1865.4904, 'train_samples_per_second': 4.824, 'train_steps_per_second': 0.603, 'train_loss': 3.748817679511176, 'epoch': 1.0})

### 9、保存 LoRA 权重

In [27]:
trainer.save_model("./mt0-lora-mac-final")

Saving model checkpoint to ./mt0-lora-mac-final
loading configuration file config.json from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/config.json
Model config MT5Config {
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "dtype": "float32",
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.57.1",
  "use_cach

生成的文件很小

```
% du -sh *
2.6M	mlruns
 22M	mt0-lora
 17M	mt0-lora-mac-final
```

## 五、测试生成的模型

加载微调后模型

In [28]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel

model_name = "bigscience/mt0-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, "./mt0-lora-mac-final")
model = model.to(device)

loading file spiece.model from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/spiece.model
loading file tokenizer.json from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/special_tokens_map.json
loading file tokenizer_config.json from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/tokenizer_config.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /Users/huhao/.cache/huggingface/hub/models--bigscience--mt0-base/snapshots/0c500042c1f80b6b6ec208cf73d9ee43f1dac245/config.js

测试生成

In [29]:
text = '''Translate English to Chinese: 
The difficult access to these older sites for displaced persons gives rise to 
concerns about the living conditions of the people who are residents there, 
in particular widows, the elderly and children, 
who often live in intolerable hardship.'''
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=1024)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

对于那些被困的人来说,这些老地方的生存状况对那些居住者来说,尤其是老人、老人和孩子,经常生活在极其困难的困难之中。
